In [1]:
import copy, glob, json, os
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_tf_cams(cam_dict, target_radius=1.):
    cam_centers = []
    for frame in cam_dict:
        C2W = np.array(frame['transform_matrix'])
        assert C2W.shape == ((4,4))
        cam_centers.append(C2W[:3, 3:4])

    def get_center_and_diag(cam_centers):
        cam_centers = np.hstack(cam_centers)
        avg_cam_center = np.mean(cam_centers, axis=1, keepdims=True)
        center = avg_cam_center
        dist = np.linalg.norm(cam_centers - center, axis=0, keepdims=True)
        diagonal = np.max(dist)
        return center.flatten(), diagonal

    center, diagonal = get_center_and_diag(cam_centers)
    radius = diagonal * 1.1

    translate = -center
    scale = target_radius / radius

    return translate, scale


def normalize_cam_dict(in_cam_dict_file, out_cam_dict_file, target_radius=1., in_geometry_file=None, out_geometry_file=None):
    with open(in_cam_dict_file) as fp:
        in_cam_dict = json.load(fp)

    translate, scale = get_tf_cams(in_cam_dict['frames'], target_radius=target_radius)

    if in_geometry_file is not None and out_geometry_file is not None:
        # check this page if you encounter issue in file io:
        # http://www.open3d.org/docs/0.9.0/tutorial/Basic/file_io.html
        geometry = o3d.io.read_triangle_mesh(in_geometry_file)
        
        tf_translate = np.eye(4)
        tf_translate[:3, 3:4] = translate
        tf_scale = np.eye(4)
        tf_scale[:3, :3] *= scale
        tf = np.matmul(tf_scale, tf_translate)

        geometry_norm = geometry.transform(tf)
        o3d.io.write_triangle_mesh(out_geometry_file, geometry_norm)
  
    def transform_pose(C2W, translate, scale):
        cam_center = C2W[:3, 3]
        C2W[:3, 3] = (cam_center + translate) * scale
        return C2W

    out_cam_dict = copy.deepcopy(in_cam_dict)
    for frame in out_cam_dict['frames']:
        C2W = np.array(frame['transform_matrix'])
        assert C2W.shape == ((4,4))
        C2W = transform_pose(C2W, translate, scale)
        assert np.isclose(np.linalg.det(C2W), 1.)
        frame['transform_matrix'] = C2W.tolist()

    with open(out_cam_dict_file, 'w') as fp:
        json.dump(out_cam_dict, fp, indent=2, sort_keys=True)

In [3]:
in_cam_dict_file = '/home/leyinghu/Documents/data/Real_data0720/agisoft_transforms_test.json'
out_cam_dict_file = os.path.join(os.path.dirname(in_cam_dict_file), 'transforms_norm.json')
normalize_cam_dict(in_cam_dict_file, out_cam_dict_file, target_radius=1.)